In [ ]:
import pandas as pd

[Fonte](https://pandas.pydata.org/docs/getting_started/intro_tutorials/08_combine_dataframes.html#)

In [ ]:
air_quality_no2 = pd.read_csv("data/air_quality_no2_long.csv",
                              parse_dates=True)


air_quality_no2 = air_quality_no2[["date.utc", "location",
                                   "parameter", "value"]]

air_quality_no2.head()

In [ ]:
air_quality_pm25 = pd.read_csv("data/air_quality_pm25_long.csv",
                               parse_dates=True)


air_quality_pm25 = air_quality_pm25[["date.utc", "location",
                                     "parameter", "value"]]


air_quality_pm25.head()

## Concatenando objetos

Quero combinar as medidas NO<sub>2</sub> e <sub>PM25</sub> de duas tabelas com estruturas semelhantes, em uma única tabela.

> A função `concat()` executa operações de concatenação de várias tabelas ao longo de um dos eixos (por linha ou por coluna).

In [ ]:
air_quality = pd.concat([air_quality_pm25, air_quality_no2], axis=0)

air_quality.head()

Por padrão, a concatenação ocorre ao longo do eixo 0, portanto, a tabela resultante combina as linhas das tabelas de entrada. Vamos verificar a forma do original e as tabelas concatenadas para verificar a operação:

In [ ]:
print('Shape of the ``air_quality_pm25`` table: ', air_quality_pm25.shape)
print('Shape of the ``air_quality_no2`` table: ', air_quality_no2.shape)
print('Shape of the resulting ``air_quality`` table: ', air_quality.shape)
print('Portanto, a tabela resultante tem 3178 = 1110 + 2068 linhas.')

> O argumento do eixo retornará em vários métodos pandas que podem ser aplicados ao longo de um eixo. O `DataFrame` tem dois eixos correspondentes: o primeiro correndo verticalmente para baixo nas linhas (eixo 0) e o segundo correndo horizontalmente nas colunas (eixo 1). A maioria das operações, como concatenação ou estatísticas resumidas, ocorre por padrão em linhas (eixo 0), mas também pode ser aplicada em colunas.

A classificação da tabela nas informações de data e hora também ilustra a combinação de ambas as tabelas, com a coluna `parameter` definindo a origem da tabela (`no2` da tabela `air_quality_no2` ou `pm25` da tabela `air_quality_pm25`):

In [ ]:
air_quality = air_quality.sort_values("date.utc")

air_quality.head()

Neste exemplo específico, a coluna `parameter` fornecida pelos dados garante que cada uma das tabelas originais possa ser identificada. Isso não é sempre o caso. A função `concat` fornece uma solução conveniente com o argumento `keys`, adicionando um índice de linha adicional (hierárquico). Por exemplo:

In [ ]:
air_quality_ = pd.concat([air_quality_pm25, air_quality_no2], keys=["PM25", "NO2"])

air_quality_.head()

In [ ]:
air_quality_.tail()

> A existência de vários índices de linha/coluna ao mesmo tempo não foi mencionada nestes tutoriais. A indexação hierárquica ou MultiIndex é um recurso de pandas avançado e poderoso para analisar dados dimensionais superiores.

> A indexação múltipla está fora do escopo desta introdução aos pandas. Por enquanto, lembre-se que a função `reset_index` pode ser usada para converter qualquer nível de um índice em uma coluna, por exemplo `air_quality.reset_index(level=0)`

> Sinta-se à vontade para mergulhar no mundo da indexação múltipla na seção do guia do usuário sobre [indexação avançada](https://pandas.pydata.org/docs/user_guide/advanced.html#advanced).

> Mais opções sobre concatenação de tabelas (linha e coluna) e como `concat` podem ser usadas para definir a lógica (união ou interseção) dos índices nos outros eixos são fornecidas na seção sobre [concatenação de objetos](https://pandas.pydata.org/docs/user_guide/merging.html#merging-concat).

## Junte tabelas usando um identificador comum

[Fonte](https://pandas.pydata.org/docs/getting_started/intro_tutorials/08_combine_dataframes.html#join-tables-using-a-common-identifier)

Adicione as coordenadas da estação, fornecidas pela tabela de metadados das estações, às linhas correspondentes na tabela de medições.

> As estações usadas neste exemplo (FR04014, BETR801 e London Westminster) são apenas três entradas listadas na tabela de metadados. Queremos apenas adicionar as coordenadas desses três à tabela de medidas, cada uma nas linhas correspondentes da tabela `air_quality`.

In [ ]:
stations_coord = pd.read_csv("data/air_quality_stations.csv")

stations_coord.head()

In [ ]:
air_quality.head()

Usando a função `merge()`, para cada uma das linhas da tabela `air_quality`, as coordenadas correspondentes são adicionadas da tabela `air_quality_stations_coord`. Ambas as tabelas possuem em comum a coluna `location` que é utilizada como chave para combinar as informações. Ao escolher a junção `left`, apenas os locais disponíveis na tabela `air_quality` (à esquerda), ou seja, FR04014, BETR801 e London Westminster, acabam na tabela resultante. A função `merge` oferece suporte a várias opções de junção semelhantes às operações de estilo de banco de dados.

In [ ]:
air_quality = pd.merge(air_quality, stations_coord, how="left", on="location")

air_quality.head()

Adicione a descrição completa e o nome dos parâmetros, fornecidos pela tabela de metadados dos parâmetros, à tabela de medições.

> Em comparação com o exemplo anterior, não há um nome de coluna comum. No entanto, a coluna `parameter` na tabela `air_quality` e a coluna `id` em `air_quality_parameters_name` ambas fornecem a variável medida em um formato comum. Os argumentos `left_one` e `right_on` são usados ​​aqui (ao invés de apenas on) para fazer o link entre as duas tabelas.

In [ ]:
air_quality_parameters = pd.read_csv("data/air_quality_parameters.csv")

air_quality_parameters.head()

In [ ]:
air_quality = pd.merge(air_quality, air_quality_parameters,
                       how='left', left_on='parameter', right_on='id')


air_quality.head()

> Pandas suporta também junções internas, externas e direitas. Mais informações sobre união/mesclagem de tabelas são fornecidas na seção do guia do usuário sobre [mesclagem de tabelas no estilo de banco de dados](https://pandas.pydata.org/docs/user_guide/merging.html#merging-join). Ou dê uma olhada na [comparação com a página SQL](https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_sql.html#compare-with-sql-join).

## LEMBRAR

* Várias tabelas podem ser concatenadas tanto em colunas quanto em linhas usando a função `concat`.

* Para mesclar/juntar tabelas como banco de dados, use a função `merge`.